## Imports

In [1]:
from classifiers import Train_SGAN_DM_Curve, Train_SGAN_Freq_Phase, Train_SGAN_Time_Phase, Train_SGAN_Pulse_Profile
from glob import glob
import pandas as pd
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

2021-10-20 13:56:49.626183: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-20 13:56:49.626211: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
base_dir = '/home/isaaccolleran/Documents/sgan/'

## Loading training, validation and unlabelled file data

In [3]:
from load_MWA_data import get_files_list, load_feature_datasets

# labelled training files
path_to_data = base_dir + 'MWA_cands/'
pfd_files, pfd_labels = get_files_list(path_to_data, 'training_labels.csv')

# validation files
path_to_validation = base_dir + 'MWA_validation/'
validation_files, validation_labels = get_files_list(path_to_validation, 'validation_labels.csv')

# unlabelled training files
path_to_unlabelled = base_dir + 'MWA_unlabelled_cands/'
unlabelled_files, unlabelled_labels = get_files_list(path_to_unlabelled, 'training_labels.csv')

# loading the physical data
dm_curve_data, freq_phase_data, pulse_profile_data, time_phase_data = load_feature_datasets(pfd_files)
validation_dm_curve_data, validation_freq_phase_data, validation_pulse_profile_data, validation_time_phase_data = load_feature_datasets(validation_files)
unlabelled_dm_curve_data, unlabelled_freq_phase_data, unlabelled_pulse_profile_data, unlabelled_time_phase_data = load_feature_datasets(unlabelled_files)

# combining labels and data
dm_curve_dataset = [dm_curve_data, pfd_labels]
dm_curve_validation_dataset = [validation_dm_curve_data, validation_labels]
dm_curve_unlabelled_dataset = [unlabelled_dm_curve_data, unlabelled_labels]


## Defining Functions (only for DM curve)
These functions will be useful for the training procedure

generate_real_samples is a function that randomly selects n_samples from the dataset. It also has the ability to apply label smoothing to the labels of the dataset

train_c_model is a function that can be called to train exclusively the supervised discriminator. It is a very specific function that will only ever needed to be called for training the c_model. All other functions, unless specified specifically, have been generalised

### Shared Functions

In [4]:
def generate_real_samples(dataset, n_samples, noisy_labels=True):
    # dataset is fed in as [x_data, labels]
    # it is also important to differentiate between labels and y values
    #    -> label = pulsar/non-pulsar
    #    -> y = real/generated
    
    # split into images and labels
    images, labels = dataset
    images = np.array(images)
    labels = np.array(labels)
    
#     print(images.shape, labels.shape)
    
    # choose random instances
    ix = np.random.randint(0, images.shape[0], n_samples)
    
    # select images and labels
    X, labels = images[ix], labels[ix]
    
    # generate class labels
    if noisy_labels:
        y = np.random.uniform(0.9, 1, (n_samples, 1))
    else:
        y = np.ones((n_samples, 1))
     
    return [X, labels], y

In [5]:
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    z_input = np.random.randn(latent_dim * n_samples)

    # reshape into a batch of inputs for the network
    z_input = z_input.reshape(n_samples, latent_dim)

    return z_input

## use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples, noisy_labels=True):
    # generate points in latent space
    z_input = generate_latent_points(latent_dim, n_samples)

    # predict outputs
    images = generator.predict(z_input)
    
    # create class labels
    if noisy_labels:
        y = np.random.uniform(0.0,0.2, (n_samples, 1))
    else:
        y = np.zeros((n_samples, 1))
    
    return images, y


In [6]:
def summarise_performance(step, c_model, validation_dataset, epoch_number, model_accuracy, save_best_model=True):
    
    validation_x, validation_y = validation_dataset
    validation_x = np.array(validation_x)
    validation_y = np.array(validation_y)
    
    _, acc = c_model.evaluate(validation_x, validation_y, verbose=0)
    
    with open('training_logs/model_performance_sgan_dm_curve.txt', 'a') as f:
        f.write('intermediate_models/dm_curve_c_model_epoch_%d.h5' % int(epoch_number) + ',' + '%.3f' % (acc) + '\n')

    if save_best_model == True:
        if acc > model_accuracy:
            print('Current Model has %.3f training accuracy which is better than previous best of %.3f. Will save it as as new best model.' % (acc * 100, model_accuracy * 100 ))
            
            filename3 = 'MWA_best_retrained_models/dm_curve_best_discriminator_model.h5'
            c_model.save(filename3)
            model_accuracy = acc
        else:
            print('Current Model is not as good as the best model. This model will not be saved. Accuracy %.3f' % (acc * 100))

        return model_accuracy, acc
    else:
        print('Classifier Accuracy: %.3f%%' % (acc * 100))

        # save the classifier model
        filename3 = 'MWA_intermediate_models/dm_curve_c_model_epoch_%d.h5' %int(epoch_number)
        c_model.save(filename3)
        print('>Saved: %s, and %s' % (filename2, filename3))
        return model_accuracy

### c_model Training Function

In [7]:
def train_c_model(c_model, dataset, validation_dataset, batch_size, latent_dim=100, n_epochs=10):
    # inputs:
    #    -> c_model: the supervised discriminator model
    #    -> dataset: numpy array containing the plot data and the labels in the form [plot_data, labels]
    #    -> validation_dataset: the same as dataset except for the validation set
    #    -> batch_size: int - number of samples before back-propagation is done
    #    -> (optional) latent_dim: int - size of sample noise for generator
    #    -> (optional) n_epoch: int - number of epochs to train for with these settings
    
    n_batch = batch_size
    x, y = dataset
    
    # number of batches per training epoch
    batch_per_epoch = int(np.array(x).shape[0] / n_batch)
    
    # calculate the number of training iterations
    n_steps = batch_per_epoch * n_epochs
    
    print('n_epochs=%d, n_batch=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, batch_per_epoch, n_steps))
    
    # initial declarations before training loop
    epoch_number = 0
    model_accuracy = 0.0
    accuracies = np.zeros((n_epochs))
    testing_acc = np.zeros((n_epochs))
    
    for i in range(n_steps):
        [x_real, y_real], _ = generate_real_samples(dataset, batch_size, noisy_labels=True)
        x_real = np.reshape(x_real, (batch_size, 60, 1))
        c_loss, c_acc = c_model.train_on_batch(x_real, y_real)
        
        # print('>%d, c[loss = %.3f, accuracy = %.0f]' % (i+1, c_loss, c_acc*100))

        # if i % batch_per_epoch == 0:
        #     print(x_real.shape, y_real.shape)
        
        # evaluate the model performance every so often
        if (i+1) % (batch_per_epoch * 1) == 0:
            epoch_number += 1
            model_accuracy, this_acc = summarise_performance(i, c_model, validation_dataset, epoch_number, model_accuracy)
            accuracies[epoch_number - 1] = this_acc

            _, test_acc = c_model.evaluate(x_real, y_real, verbose=0)
            testing_acc[epoch_number - 1] = test_acc

            
    
    # plotting the accuracies
    plt.figure()
    plt.plot(list(range(n_epochs)), accuracies, list(range(n_epochs)), testing_acc)
    plt.xlabel('Epoch no.')
    plt.ylabel('Accuracy')
    plt.title('Batch size = {}, num epochs = {}'.format(batch_size, n_epochs))
    plt.legend(['validation', 'training'])
    plt.savefig('retrain_sgan.png')


## Creating Model Instances

In [8]:
batch_size = 16

dm_curve_instance = Train_SGAN_DM_Curve(dm_curve_data, pfd_labels, validation_dm_curve_data, validation_labels, unlabelled_dm_curve_data, unlabelled_labels, batch_size)
pulse_profile_instance = Train_SGAN_Pulse_Profile(pulse_profile_data, pfd_labels, validation_pulse_profile_data, validation_labels, unlabelled_pulse_profile_data, unlabelled_labels, batch_size)
freq_phase_instance = Train_SGAN_Freq_Phase(freq_phase_data, pfd_labels, validation_freq_phase_data, validation_labels, unlabelled_freq_phase_data, unlabelled_labels, batch_size)
time_phase_instance = Train_SGAN_Time_Phase(time_phase_data, pfd_labels, validation_time_phase_data, validation_labels, unlabelled_time_phase_data, unlabelled_labels, batch_size)

### Retraining from scratch

In [9]:
# retraining freq_phase model ##################
# d_model, c_model = freq_phase_instance.define_discriminator()
# generator = freq_phase_instance.define_generator()
# gan = freq_phase_instance.define_gan(generator, d_model)
# freq_phase_instance.train(generator, d_model, c_model, gan, n_epochs=25)

# retraining time_phase model ##################
# d_model, c_model = time_phase_instance.define_discriminator()
# generator = time_phase_instance.define_generator()
# gan = time_phase_instance.define_gan(generator, d_model)
# time_phase_instance.train(generator, d_model, c_model, gan, n_epochs=25)

# retraining dm_curve model ####################
# d_model, c_model = dm_curve_instance.define_discriminator()
# generator = freq_phase_instance.define_generator()
# gan = freq_phase_instance.define_gan(generator, d_model)
# freq_phase_instance.train(generator, d_model, c_model, gan, n_epochs=25)

# retraining pulse_profile model ################
# d_model, c_model = pulse_profile_instance.define_discriminator()
# generator = freq_phase_instance.define_generator()
# gan = freq_phase_instance.define_gan(generator, d_model)
# freq_phase_instance.train(generator, d_model, c_model, gan, n_epochs=25)

In [10]:
model = load_model('MWA_best_retrained_models/pre-trained/dm_curve_best_discriminator_model.h5')
# print(model.summary())
a = model.predict(validation_dm_curve_data)

2021-10-20 13:59:20.594683: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-20 13:59:20.613855: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3193480000 Hz


In [11]:
print(a)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [11]:
# import matplotlib.pyplot as plt
# 
# print(freq_phase_data.shape)
# fig = plt.figure()
# plt.imshow(freq_phase_data[4], cmap='gray')

# freq_phase_instance.train(generator, d_model, c_model, gan, n_epochs=25)

### Creating GAN

In [19]:
dir_to_model = base_dir + 'best_retrained_models/'

# loading generators
dm_curve_generator = load_model(dir_to_model + 'dm_curve_best_generator_model.h5')
time_phase_generator = load_model(dir_to_model + 'time_phase_best_generator_model.h5')
freq_phase_generator = load_model(dir_to_model + 'freq_phase_best_generator_model.h5')
pulse_profile_generator = load_model(dir_to_model + 'pulse_profile_best_generator_model.h5')

# creating d_model discriminators
dm_curve_discriminator, _ = dm_curve_instance.define_discriminator()
pulse_profile_discriminator, _ = pulse_profile_instance.define_discriminator()
time_phase_discriminator, _ = time_phase_instance.define_discriminator()
freq_phase_discriminator, _ = freq_phase_instance.define_discriminator()

# loading the saved weights
dm_curve_discriminator.load_weights(dir_to_model + 'dm_curve_best_discriminator_model.h5')
pulse_profile_discriminator.load_weights(dir_to_model + 'pulse_profile_best_discriminator_model.h5')
time_phase_discriminator.load_weights(dir_to_model + 'time_phase_best_discriminator_model.h5')
freq_phase_discriminator.load_weights(dir_to_model + 'freq_phase_best_discriminator_model.h5')

# creating the GANs
dm_curve_gan = dm_curve_instance.define_gan(dm_curve_generator, dm_curve_discriminator)
pulse_profile_gan = pulse_profile_instance.define_gan(pulse_profile_generator, pulse_profile_discriminator)
time_phase_gan = freq_phase_instance.define_gan(time_phase_generator, time_phase_discriminator)
freq_phase_gan = time_phase_instance.define_gan(freq_phase_generator, freq_phase_discriminator)

# setting all layers except the dense layers to not trainable
for l in (dm_curve_discriminator.layers + time_phase_discriminator.layers + freq_phase_discriminator.layers + pulse_profile_discriminator.layers):
# for loop combines all the layers into one so that we dont need 4 separate loops
    if not l.name.startswith('dense'):
        l.trainable = False
    # print(l.name, l.trainable)

### Loading Pretrained Discrimators

Within this cell, all of the layers that aren't dense layers are changed to not trainable. We do this because we only want to update the weights of the densely connected layers.

In [20]:
dir_to_model = base_dir + 'best_retrained_models/'

# loading the models from the above
dm_curve_model = load_model(dir_to_model + 'dm_curve_best_discriminator_model.h5')
# dm_curve_model = load_model(base_dir + 'MWA_best_retrained_models/93.3_learn2e-5beta0.99_dm_curve_best_discriminator_model.h5')
time_phase_model = load_model(dir_to_model + 'time_phase_best_discriminator_model.h5')
freq_phase_model = load_model(dir_to_model + 'freq_phase_best_discriminator_model.h5')
pulse_profile_model = load_model(dir_to_model + 'pulse_profile_best_discriminator_model.h5')

# setting all layers except the dense layers to not trainable
for l in (dm_curve_model.layers + time_phase_model.layers + freq_phase_model.layers + pulse_profile_model.layers):
# for loop combines all the layers into one so that we dont need 4 separate loops
    if not l.name.startswith('dense'):
        l.trainable = False
    # print(l.name, l.trainable)


### Updating optimisers for the models

In [21]:
opt1 = tf.optimizers.Adam(learning_rate=5e-2, beta_1=0.999) # c discriminator
# opt1 = tf.optimizers.Adam(learning_rate=0.01, beta_1=0.99) # c discriminator
opt2 = tf.optimizers.Adam(learning_rate=5e-2, beta_1=0.999) # d discriminator
opt3 = tf.optimizers.Adam(learning_rate=5e-2, beta_1=0.999) # gan

# dm_curve_model.optimizer = opt1
# dm_curve_discriminator.optimizer = opt2
# dm_curve_gan.optimizer = opt3

freq_phase_model.optimizer = opt1
freq_phase_discriminator.optimizer = opt2
freq_phase_gan.optimizer = opt3

# dm_curve_model.optimizer = opt1
# dm_curve_discriminator.optimizer = opt2
# dm_curve_gan.optimizer = opt3

# dm_curve_model.optimizer = opt1
# dm_curve_discriminator.optimizer = opt2
# dm_curve_gan.optimizer = opt3

In [22]:
# print(dm_curve_model.optimizer.learning_rate, dm_curve_model.optimizer.beta_1)

# _, acc = dm_curve_model.evaluate(validation_dm_curve_data, validation_labels)
# print(acc)
# print(dm_curve_model.metrics_names)
# predictions = dm_curve_model.predict(validation_dm_curve_data)
# print(np.sum(np.sum(predictions, axis=1)))
# print(predictions)
# after = np.rint(predictions)
# after = np.argmax(after, axis=1)
# after = np.reshape(after, len(after))
# print([after, validation_labels])

In [23]:
# print(dm_curve_dataset[0].shape, dm_curve_dataset[1].shape)
# this = dm_curve_dataset[0][413, :]

# plt.figure()
# plt.plot(this)

## Calling Training Function

In [24]:
batch_size = 16
n_epoch = 25

# train_c_model(dm_curve_model, dm_curve_dataset, dm_curve_validation_dataset, batch_size, n_epochs=n_epoch)
# dm_curve_instance.train(dm_curve_generator, dm_curve_discriminator, dm_curve_model, dm_curve_gan, n_epochs=n_epoch, noisy_labels=True)
freq_phase_instance.train(freq_phase_generator, freq_phase_discriminator, freq_phase_model, freq_phase_gan, n_epochs=n_epoch, noisy_labels=True)
# pulse_profile_instance.train(pulse_profile_generator, pulse_profile_discriminator, pulse_profile_model, pulse_profile_gan, n_epochs=n_epoch, noisy_labels=True)
# time_phase_instance.train(time_phase_generator, time_phase_discriminator, time_phase_model, time_phase_gan, n_epochs=n_epoch, noisy_labels=True)

batch per epoch is 251
n_epochs=25, n_batch=16, 1/2=8, b/e=251, steps=6275
Current Model has 30.000 training accuracy which is better than previous best of 0.000. Will save it as as new best model.
Current Model has 88.333 training accuracy which is better than previous best of 30.000. Will save it as as new best model.
Current Model is not as good as the best model. This model will not be saved. Accuracy 25.000
Current Model is not as good as the best model. This model will not be saved. Accuracy 38.333
Current Model is not as good as the best model. This model will not be saved. Accuracy 13.333
Current Model is not as good as the best model. This model will not be saved. Accuracy 41.667
Current Model is not as good as the best model. This model will not be saved. Accuracy 11.667
Current Model is not as good as the best model. This model will not be saved. Accuracy 43.333
Current Model is not as good as the best model. This model will not be saved. Accuracy 30.000
Current Model is not